# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/powertothefuture/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/powertothefuture/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/powertothefuture/Documents/aimakerspace/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/powertothefuture/Documents/aimakerspace/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/powertothefuture/Documents/aimakerspace/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node 'ad687e'. Skipping!
Property 'summary' already exists in node 'bb46d7'. Skipping!
Property 'summary' already exists in node '0b2657'. Skipping!
Property 'summary' already exists in node '158072'. Skipping!
Property 'summary' already exists in node '8fda48'. Skipping!
Property 'summary' already exists in node 'dd4d78'. Skipping!
Property 'summary' already exists in node 'a60c37'. Skipping!
Property 'summary' already exists in node '397001'. Skipping!
Property 'summary' already exists in node 'e74f72'. Skipping!
Property 'summary' already exists in node '89e202'. Skipping!
Property 'summary' already exists in node '4e32d6'. Skipping!
Property 'summary' already exists in node '60c442'. Skipping!
Property 'summary' already exists in node 'aa3b9d'. Skipping!
Property 'summary' already exists in node 'cd009c'. Skipping!
Property 'summary' already exists in node 'a8108a'. Skipping!
Property 'summary' already exists in node '236ab8'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '8fda48'. Skipping!
Property 'summary_embedding' already exists in node 'bb46d7'. Skipping!
Property 'summary_embedding' already exists in node 'ad687e'. Skipping!
Property 'summary_embedding' already exists in node 'dd4d78'. Skipping!
Property 'summary_embedding' already exists in node 'a60c37'. Skipping!
Property 'summary_embedding' already exists in node '0b2657'. Skipping!
Property 'summary_embedding' already exists in node '397001'. Skipping!
Property 'summary_embedding' already exists in node '158072'. Skipping!
Property 'summary_embedding' already exists in node 'e74f72'. Skipping!
Property 'summary_embedding' already exists in node 'aa3b9d'. Skipping!
Property 'summary_embedding' already exists in node '89e202'. Skipping!
Property 'summary_embedding' already exists in node '9795b1'. Skipping!
Property 'summary_embedding' already exists in node '4e32d6'. Skipping!
Property 'summary_embedding' already exists in node 'cd009c'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 710)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 710)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


##### ✅ Answer:
SingleHopSpecificQuerySynthesizer:  questions that need only straight forward information from one single piece of information such as who is author of the document? What is the main topic? What is the capital of France etc..

MultiHopAbstractQuerySynthesizer :  questions that require combining information from multiple sources that require reasoning Examples: What are over all implications of AI? How did the ChatGPT launch and adaptation lead to global diffusion? etc..

MultiHopSpecificQuerySynthesizer : Questions that require specific facts from multiple sources, combines them to anwer Example: What is the important in November 2022 and November 2024? How do the total messages sent by ChatGPT users vary between work and non-work usage? etc..

Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Wha is the signifcance of Novmber 2022 in the ...,[Introduction ChatGPT launched in November 202...,Introduction ChatGPT launched in November 2022.,single_hop_specifc_query_synthesizer
1,What is the significance of June 2024 in the c...,[Table 1: ChatGPT daily message counts (millio...,The report provides data on ChatGPT daily mess...,single_hop_specifc_query_synthesizer
2,How do professional occupations influence the ...,[Variation by Occupation Figure 23 presents va...,Users in highly paid professional and technica...,single_hop_specifc_query_synthesizer
3,What is the significance of July 2025 in the c...,[Conclusion This paper studies the rapid growt...,"By July 2025, ChatGPT had been used weekly by ...",single_hop_specifc_query_synthesizer
4,how data privacy and dataset limits affect Cha...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context explains that variation in ChatGPT...,multi_hop_abstract_query_synthesizer
5,Based on the data showing that non-work messag...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data indicates that non-work messages have...,multi_hop_abstract_query_synthesizer
6,how ChatGPT launch and the global diffusion of...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,ChatGPT was launched in November 2022 and by J...,multi_hop_abstract_query_synthesizer
7,how 700 million users use chatgpt for work and...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"According to the context, by July 2025, ChatGP...",multi_hop_specific_query_synthesizer
8,How do the total messages sent by ChatGPT user...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The context indicates that by July 2025, ChatG...",multi_hop_specific_query_synthesizer
9,wha is the impoortant in November 2022 and Nov...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The context indicates that ChatGPT was launche...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node 'e736cb'. Skipping!
Property 'summary' already exists in node '23642b'. Skipping!
Property 'summary' already exists in node '72ed72'. Skipping!
Property 'summary' already exists in node 'f9261d'. Skipping!
Property 'summary' already exists in node 'ee3efd'. Skipping!
Property 'summary' already exists in node '1d4cb4'. Skipping!
Property 'summary' already exists in node '876961'. Skipping!
Property 'summary' already exists in node '2432c6'. Skipping!
Property 'summary' already exists in node 'ce0af5'. Skipping!
Property 'summary' already exists in node 'dfdbdb'. Skipping!
Property 'summary' already exists in node '70e43b'. Skipping!
Property 'summary' already exists in node '56172e'. Skipping!
Property 'summary' already exists in node '5d1c3d'. Skipping!
Property 'summary' already exists in node '09d114'. Skipping!
Property 'summary' already exists in node 'c41932'. Skipping!
Property 'summary' already exists in node '457255'. Skipping!
Property

...

Property 'summary_embedding' already exists in node '1d4cb4'. Skipping!
Property 'summary_embedding' already exists in node 'f9261d'. Skipping!
Property 'summary_embedding' already exists in node '23642b'. Skipping!
Property 'summary_embedding' already exists in node 'ee3efd'. Skipping!
Property 'summary_embedding' already exists in node '72ed72'. Skipping!
Property 'summary_embedding' already exists in node 'e736cb'. Skipping!
Property 'summary_embedding' already exists in node '70e43b'. Skipping!
Property 'summary_embedding' already exists in node '876961'. Skipping!
Property 'summary_embedding' already exists in node '2432c6'. Skipping!
Property 'summary_embedding' already exists in node '56172e'. Skipping!
Property 'summary_embedding' already exists in node 'ce0af5'. Skipping!
Property 'summary_embedding' already exists in node 'dfdbdb'. Skipping!
Property 'summary_embedding' already exists in node '5d1c3d'. Skipping!
Property 'summary_embedding' already exists in node '09d114'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"How many users are using ChatGPT, reaching 700...",[Introduction ChatGPT launched in November 202...,"By July 2025, 700 million users were sending a...",single_hop_specifc_query_synthesizer
1,"So like in Jun 2025, how many total messages w...",[Month Non-Work (M) (%) Work (M) (%) Total Mes...,"In June 2025, the total messages sent were 2,6...",single_hop_specifc_query_synthesizer
2,What is the role of OpenAI in AI development?,[Total daily counts are exact measurements of ...,Total daily counts are exact measurements of m...,single_hop_specifc_query_synthesizer
3,Panel A what does it show about ChatGPT work s...,[Figure 23 presents variation in ChatGPT usage...,Figure 23 shows that unadjusted work shares ar...,single_hop_specifc_query_synthesizer
4,How does the variation in ChatGPT usage by occ...,"[<1-hop>\n\n6.5 Variation by Occupation, <2-ho...",Figure 23 illustrates the variation in ChatGPT...,multi_hop_abstract_query_synthesizer
5,Hw did the ChatGPT launch and adptation lead t...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The introduction of ChatGPT in November 2022 a...,multi_hop_abstract_query_synthesizer
6,"How does ChatGPT's work use, like writing and ...",[<1-hop>\n\nConclusion This paper studies the ...,"ChatGPT's work-related usage, especially writi...",multi_hop_abstract_query_synthesizer
7,so tell me how ChatGPT use by occupation like ...,[<1-hop>\n\nFigure 23 presents variation in Ch...,Figure 23 shows that users in highly paid prof...,multi_hop_abstract_query_synthesizer
8,How does the growth in the total number of mes...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"By July 2025, ChatGPT users were collectively ...",multi_hop_specific_query_synthesizer
9,how much messages in june 2024 and june 2025 a...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"In June 2024, there were 238 million non-work ...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [39]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [ ]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [ ]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [ ]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [ ]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [ ]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'People are using AI, particularly generative AI like ChatGPT, in many different ways both at work and outside of work. At work, AI is used to perform workplace tasks by augmenting or automating human labor, improving productivity either as a co-worker producing output or as a co-pilot giving advice. Users employ AI to produce writing, software code, spreadsheets, and other digital products, distinguishing generative AI from traditional web search engines. Outside of work, AI is used for self-expression, relationships, personal reflection, games, and role play, though these use cases are smaller in proportion. Overall, generative AI is flexible and is being adopted rapidly for tasks involving information seeking, advice, creative production, and workplace assistance.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [ ]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dopeness_evaluator`:

##### ✅ Answer:
qa_evaluator enables basic question answering whether RAG system anwering accurately with the given information

labeled_helpfullness_evaluator whether the generated answer is helpful to the user or not. If the answer is technically right, whether that is useful to the user validating whether the reference is considered or not

dopeness_evaluator evaulates for whether answer is generic response or it is engaging. 



## LangSmith Evaluation

In [ ]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'cold-wood-51' at:
https://smith.langchain.com/o/579c282e-d619-4808-8787-a7f5dcb91419/datasets/7157f26d-e061-4d7d-83c4-81bbf4c5bb7e/compare?selectedSessions=fa52da97-ea20-47e5-9b01-439ffdf95f1c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Jun 2025 and July 2025 ChatGPT use how much me...,"Based on the context provided, the data for Ju...",None,"In June 2025, there were 238 million non-work ...",0,0,0,2.518508,a7db9563-a91e-4a40-9bba-a9d2e79269bb,44097dc3-97e9-4ec7-b502-97707888bd63
1,so in july 2025 and jun 2025 how many messages...,"In June 2025, a total of 2,627 million (2.627 ...",None,"In June 2025, there were a total of 2,627 mill...",0,0,0,3.427021,7c18226f-ae19-42ea-aa0b-0739b3b2620b,3c687b87-4fa6-43cb-9332-3f75e386776c
2,how much messages in june 2024 and june 2025 a...,Based on the provided context:\n\n- In June 20...,None,"In June 2024, there were 238 million non-work ...",1,1,0,5.712126,8070f274-d318-41fb-93b2-599f540f47a5,d26f8eee-f857-4762-8d2c-cc67b8a15536
3,How does the growth in the total number of mes...,"By July 2025, ChatGPT had more than 700 millio...",None,"By July 2025, ChatGPT users were collectively ...",1,1,0,2.249222,5a345b57-a5d3-4044-a09d-bb921ed5aea2,98b057d5-ec4a-48d1-ab9b-7f0fbc8c12bc
4,so tell me how ChatGPT use by occupation like ...,Based on the provided context:\n\n1. **ChatGPT...,None,Figure 23 shows that users in highly paid prof...,1,1,0,9.844268,1818cd75-1997-44bc-9304-93fe35350cd4,b41de177-88ab-430e-b28f-664b33db4d2a
5,"How does ChatGPT's work use, like writing and ...","Based on the provided context, ChatGPT users' ...",None,"ChatGPT's work-related usage, especially writi...",1,0,0,3.884952,4fd7212a-ab82-47d4-bf85-c36c4513ed34,d81d9d24-3dd8-4907-9219-cb99633ba9e2
6,Hw did the ChatGPT launch and adptation lead t...,"Based on the provided context, the launch and ...",None,The introduction of ChatGPT in November 2022 a...,1,1,0,4.429461,b24aab73-375b-4226-963f-059f352216f5,70abdcf7-e5b1-40a1-b215-2f26fdaf68db
7,How does the variation in ChatGPT usage by occ...,Based on the provided context from Figure 23 a...,None,Figure 23 illustrates the variation in ChatGPT...,1,1,0,9.103494,f20dc662-1c43-4598-a2a8-9a9ad70e43e1,b7b6710f-17f0-4034-8a83-bc6d399df717
8,Panel A what does it show about ChatGPT work s...,Based on the provided context:\n\n- **Panel A ...,None,Figure 23 shows that unadjusted work shares ar...,1,1,0,7.677392,7012eafa-2c76-4070-b147-742ab2a7543c,8e6e888a-c897-42c5-aa11-bdf3f1a51c96
9,What is the role of OpenAI in AI development?,"Based on the provided context, OpenAI's role i...",None,Total daily counts are exact measurements of m...,0,1,0,2.327706,318a23da-266f-4d67-816b-59bbd0efdc81,36c94245-fa58-4ca9-b2cb-265193372f90


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [ ]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [ ]:
rag_documents = docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:
The chunk_size=500 probably fragmenting infromation across multiples pieces, making it harder for the retriever to find complete context and for the LLM to generate coherent answers. This leads to incomplete responses, especially for complex questions that require combining information from different parts of a document. Increasing chunk_size to 1000 allows retriever to capture more comprehensive context in fewer pieces so it leads to better answer completeness and improved quality particularly for multi-hop query synthesizer questions.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:
Better embedding models like text-embedding-3-large compared to text-embedding-3-small,  captures more semantic relationships and context, leading to more accurate document retrieval. This means the RAG system finds more relevant chunks for user questions, resulting in better answer quality and completeness.

For a question like "How does ChatGPT usage vary by occupation?", a better embedding model would relate to work, professional occupations, and also various job related tasks  with the usage adoption or engagement of ChatGPT at their work related tasks.

In [ ]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [ ]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [ ]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [ ]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, let’s crank this up to eleven! People are raking in the green by harnessing AI, especially ChatGPT, not just as a mindless task robot but as a slick advisor and research sidekick. According to the cosmic wisdom dropped in the context, AI’s flex is in *boosting decision-making quality*—think knowledge-heavy gigs where crisp, sharp choices push productivity through the roof. \n\nWorkers ain’t handing over their jobs to AI; instead, they’re teaming up, letting AI pour the brain juice for smarter moves, which means more cash flow and mindshare value. Plus, the economic spike is jaw-dropping: users literally value AI so much they’d shell out $98 monthly to NOT lose it—hello, that’s a consumer surplus north of $97 billion yearly in the US alone.\n\nIn essence, the money magic happens because AI is the ultimate decision-support wingman, turbocharging work output in *knowledge-intensive* arenas. So, people cash in by riding AI’s genius to make sharper calls, power up their hustle, an

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'cold-need-13' at:
https://smith.langchain.com/o/579c282e-d619-4808-8787-a7f5dcb91419/datasets/7157f26d-e061-4d7d-83c4-81bbf4c5bb7e/compare?selectedSessions=c26c949b-a51f-4a7c-b30f-aa1fddfa9fa9




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Jun 2025 and July 2025 ChatGPT use how much me...,"Yo, check this out — in June 2025, ChatGPT was...",None,"In June 2025, there were 238 million non-work ...",0,0,1,3.034247,a7db9563-a91e-4a40-9bba-a9d2e79269bb,762bb9e6-8e93-4ea0-a3e3-d4ce31bf1016
1,so in july 2025 and jun 2025 how many messages...,"Alright, let’s blast off with the stats and th...",None,"In June 2025, there were a total of 2,627 mill...",1,0,1,4.402320,7c18226f-ae19-42ea-aa0b-0739b3b2620b,3edb791d-b3c1-4a4d-b911-46cb7cba7740
2,how much messages in june 2024 and june 2025 a...,"Alright, buckle up for some next-level ChatGPT...",None,"In June 2024, there were 238 million non-work ...",1,1,1,4.193661,8070f274-d318-41fb-93b2-599f540f47a5,0483f8b7-d583-45c5-8b47-e8cbb432ff90
3,How does the growth in the total number of mes...,"Whoa, buckle up for this AI rollercoaster! The...",None,"By July 2025, ChatGPT users were collectively ...",1,1,1,4.371757,5a345b57-a5d3-4044-a09d-bb921ed5aea2,1a194305-63f7-44dd-bb19-99fb8336d356
4,so tell me how ChatGPT use by occupation like ...,"Alright, buckle up for the ultimate deep dive ...",None,Figure 23 shows that users in highly paid prof...,1,0,1,9.931955,1818cd75-1997-44bc-9304-93fe35350cd4,d3f12d71-45fb-41db-81ab-f0d40ca499ee
5,"How does ChatGPT's work use, like writing and ...","Alright, let’s slice through the noise and dro...",None,"ChatGPT's work-related usage, especially writi...",1,1,1,6.448466,4fd7212a-ab82-47d4-bf85-c36c4513ed34,7ac0540e-6bfe-4742-b6ff-15c640200cfd
6,Hw did the ChatGPT launch and adptation lead t...,"Yo, buckle up—this is how the ChatGPT launch f...",None,The introduction of ChatGPT in November 2022 a...,1,1,1,5.128967,b24aab73-375b-4226-963f-059f352216f5,42f550b0-ed77-4fc1-913b-e8abcc974387
7,How does the variation in ChatGPT usage by occ...,"Listen up, because the breakdown of ChatGPT’s ...",None,Figure 23 illustrates the variation in ChatGPT...,1,1,1,9.227516,f20dc662-1c43-4598-a2a8-9a9ad70e43e1,bd6b3c02-0af6-4e9e-b7f6-e98105963af8
8,Panel A what does it show about ChatGPT work s...,"Alright, buckle up — here’s the ultimate break...",None,Figure 23 shows that unadjusted work shares ar...,1,1,1,7.846040,7012eafa-2c76-4070-b147-742ab2a7543c,a1c4097d-bb71-4c4d-8d69-f8e0539b25b8
9,What is the role of OpenAI in AI development?,"Yo, from the context laid out, OpenAI is strai...",None,Total daily counts are exact measurements of m...,0,0,1,2.922134,318a23da-266f-4d67-816b-59bbd0efdc81,799c883a-ef78-425e-b96e-cc41bf15a96f


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### ✅ Answer:

![Evaluation 1 Results](./Evaluation1.png "Evaluation 1 - Default Chain Results with chunk-size 500, embedding model = text-embedding-3-small")

![Evaluation 2 Results](./Evaluation2.png "Evaluation 2 - Dopeness Chain Results with chunk-size 1000, embedding_model = text-embedding-3-large")



## Observations

Dopeness is significantly better	in the evaluation because of direct prompt instruction for engagement

QA	is improved moderately because of chunk size increase enabling better retrieval from larger chunks + better embeddings

Helpfulness	is slightly reduced could be because of the "dopeness" prompt additions, the engagement is prioritized over the completeness , useful and clear answers